In [15]:
import pandas as pd
import numpy as np

In [75]:
df_steam_user = pd.read_csv(
    '../../data/raw/steam_video_games.csv',
    sep=',',
    header=None,
    names=['USER_ID', 'GAME_TITLE', 'BEHAVIOR_NAME', 'VALUE', '0']
)

df_steam_games = pd.read_csv(
    '../../data/raw/steam_games_dataset.csv',
    usecols=['Name', 'Required age', 'Price', 'Windows', 'Mac', 'Linux', 'Achievements', 'Developers', 'Categories', 'Genres']
)

In [76]:
pd.options.display.max_columns = 200

In [77]:
def cria_identificadores_genero(df:pd.DataFrame, col:str, sep:str = ',') -> pd.DataFrame:
    """
    Função que cria identificadores para jogos a partir de colunas contendo strings separadas por um delimitador comum
    """
    df = df.copy()

    generos = np.unique(
        df[col]
        .drop_duplicates()
        .str.cat(sep=sep)
        .split(sep)
    )

    for genero in generos:
        df[f'{genero}'.upper()] = (
            df[col].str.contains(genero, regex=True)
        )

    df = df.drop(col, axis=1)
    return df

In [79]:
df_steam_full = df_steam_user.merge(df_steam_games, left_on='GAME_TITLE', right_on='Name', how='inner').drop('Name', axis=1)

In [80]:
df_steam_full

,USER_ID,GAME_TITLE,BEHAVIOR_NAME,VALUE,0,Required age,Price,Windows,Mac,Linux,Achievements,Developers,Categories,Genres
0,151603712,Fallout 4,purchase,1.0,0,17,19.99,True,False,False,84,Bethesda Game Studios,"Single-player,Steam Achievements,Full controll...",RPG
1,151603712,Fallout 4,play,87.0,0,17,19.99,True,False,False,84,Bethesda Game Studios,"Single-player,Steam Achievements,Full controll...",RPG
2,151603712,Left 4 Dead 2,purchase,1.0,0,0,9.99,True,True,True,101,Valve,"Single-player,Multi-player,PvP,Online PvP,Co-o...",Action
3,151603712,Left 4 Dead 2,play,8.9,0,0,9.99,True,True,True,101,Valve,"Single-player,Multi-player,PvP,Online PvP,Co-o...",Action
4,151603712,HuniePop,purchase,1.0,0,18,9.99,True,True,True,20,HuniePot,"Single-player,Steam Achievements,Steam Trading...","Indie,RPG,Simulation,Strategy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98540,128470551,Titan Souls,play,1.5,0,0,14.99,True,True,False,27,Acid Nerve,"Single-player,Steam Achievements,Full controll...","Action,Adventure,Indie"
98541,128470551,RUSH,purchase,1.0,0,0,4.99,True,False,False,16,The Binary Mill,"Single-player,Multi-player,PvP,Online PvP,Cros...","Action,Racing,Simulation,Sports"
98542,128470551,RUSH,purchase,1.0,0,0,0.99,True,True,True,25,Two Tribes,"Single-player,Steam Achievements,Steam Cloud","Casual,Indie,Strategy"
98543,128470551,RUSH,play,1.4,0,0,4.99,True,False,False,16,The Binary Mill,"Single-player,Multi-player,PvP,Online PvP,Cros...","Action,Racing,Simulation,Sports"
